<a href="https://colab.research.google.com/github/anjaleeDS/Sandbox/blob/master/spacy_ner_satanic_bible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import spacy
from spacy import displacy
import pandas as pd
import altair as alt

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive
%cd My\ Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
/gdrive/My Drive


In [4]:
myfile = 'TSB.txt'
f = open(myfile, "rb")
text_read = f.read().decode(errors='replace')
# text_read = text.encode('ascii',errors='ignore')
text = str(text_read)
text = text.replace('\t','')
text = text.replace('\n','')
text = text.replace('\r','')

In [23]:
# print((text))
!!python -m spacy download en_core_web_sm



['Requirement already satisfied: en_core_web_sm==2.2.5 from https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz#egg=en_core_web_sm==2.2.5 in /usr/local/lib/python3.6/dist-packages (2.2.5)',
 'Requirement already satisfied: spacy>=2.2.2 in /usr/local/lib/python3.6/dist-packages (from en_core_web_sm==2.2.5) (2.2.4)',
 'Requirement already satisfied: srsly<1.1.0,>=1.0.2 in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.0.2)',
 'Requirement already satisfied: setuptools in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (49.1.0)',
 'Requirement already satisfied: blis<0.5.0,>=0.4.0 in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (0.4.1)',
 'Requirement already satisfied: wasabi<1.1.0,>=0.4.0 in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (0.7.1)',
 'Requirement already satisfied: catalogue

In [24]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)


In [ ]:
list_text = []
list_ent = []

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    list_text.append(ent.text)
    list_ent.append(ent.label_)


In [63]:
text_df = pd.DataFrame(list_text, columns=['text'])
text_df['ent'] = list_ent
text_df

,text,ent
0,The Black Pope,WORK_OF_ART
1,Anton LaVey,PERSON
2,High Priesthood of the Church of Satan,WORK_OF_ART
3,only 16 years old,DATE
4,Saturday,DATE
...,...,...
3968,American,NORP
3969,Avon,ORG
3970,The more I study religions,WORK_OF_ART
3971,Richard F. Burton,PERSON


In [62]:
#graph only the top 100 entries. get the most occuring text

text_df.text.unique()

# subset_test_df = test_df

array(['The Black Pope', 'Anton LaVey',
       'High Priesthood of the Church of Satan', ...,
       'The more I study religions', 'Richard F. Burton', '1812-1890'],
      dtype=object)

In [56]:
#graph it
bars = alt.Chart(subset_test_df).mark_bar().encode(
    x='count(ent):Q',
    y=alt.Y('text:N', sort='x'),
    tooltip=('ent:N', 'count(ent):Q')
).interactive()


text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='ent:Q'
)

bars.properties(height=900)

alt.Chart(...)